In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch.optim as opt
import time

In [ ]:
start = time.time()
torch.cuda.is_available()

In [ ]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu" 
    
device = torch.device(dev)

In [ ]:
device

In [ ]:
#df = pd.read_excel(r'/kaggle/input/distribution-coefficient-data/Filled data.xlsx')
dfs = pd.read_excel(r'/kaggle/input/simulation/SEESPEC_DATA_U6 - Copy.xlsx')


In [ ]:
df2 = dfs[['U-AQU','H-AQU','D-H','D-U']]
df3 = df2.reset_index(drop=True)
#df3 = pd.concat([df1,df2],axis=0)

In [ ]:
df3.head

In [ ]:
df3['U-AQU']= (df3['U-AQU'])*(1+df3['D-U'])
df3['H-AQU']= (df3['H-AQU'])*(1+df3['D-H'])
df3.head

In [ ]:
def normalised(database):
    mean = np.mean(database, axis=0)
    std = np.std(database, axis=0)
    return((database-mean)/std)

In [ ]:
def stand(database):
    max = np.max(database, axis=0)
    min = np.min(database, axis=0)
    return((database-min)/(max-min))

In [ ]:
print(f"max {np.max(df3['D-H'])} | min {np.min(df3['D-H'])}")

In [ ]:
df3.loc[:,['U-AQU']] = normalised(df3.loc[:,['U-AQU']])
df3.loc[:,['H-AQU']] = normalised(df3.loc[:,['H-AQU']])
df3.loc[:,['D-U']] = stand(df3.loc[:,['D-U']])
df3.loc[:,['D-H']] = stand(df3.loc[:,['D-H']])

In [ ]:
df3.head

In [ ]:
Train , Test = train_test_split(df3, test_size=0.2, random_state=69)
Train_in = Train.loc[:,['U-AQU','H-AQU']]
Test_in = Test.loc[:,['U-AQU','H-AQU']]
Train_out = Train.loc[:,['D-U','D-H']]
Test_out = Test.loc[:,['D-U','D-H']]


In [ ]:
#converting to Tensors
xtr = torch.tensor(Train_in.values, dtype=torch.float32)
xte = torch.tensor(Test_in.values, dtype=torch.float32)
ytr = torch.tensor(Train_out.values, dtype=torch.float32)
yte = torch.tensor(Test_out.values, dtype=torch.float32)

In [ ]:
xtr = xtr.to(device)
xte = xte.to(device)
ytr = ytr.to(device)
yte = yte.to(device)

In [ ]:
xtr.device

In [ ]:
class twolayermodel(nn.Module):
    def __init__(self,l1,l2):
        super().__init__()
        self.layer1 = nn.Linear(2,l1)
        #self.layer2 = nn.Linear(10,7)
        self.layer3 = nn.Linear(l1,l2)
        self.layer4 = nn.Linear(l2,2)
            
    def forward(self,x):
        y = torch.tanh(self.layer1(x))
        #y = torch.tanh(self.layer2(y))
        y = torch.tanh(self.layer3(y))
        y = self.layer4(y)
        return y

In [ ]:
class onelayermodel(nn.Module):
    def __init__(self, nodes):
        super().__init__()
        self.layer1 = nn.Linear(2,nodes)
        #self.layer2 = nn.Linear(10,7)
        #self.layer3 = nn.Linear(5,3)
        self.layer4 = nn.Linear(nodes,2)
            
    def forward(self,x):
        y = torch.tanh(self.layer1(x))
        #y = torch.tanh(self.layer2(y))
        #y = torch.tanh(self.layer3(y))
        y = self.layer4(y)
        return y

In [ ]:
torch.manual_seed(123)
lrate = [0.1,0.001]

n_iters = [10000,15000,20000,25000]
nodes= [5,7,10]

func =[opt.Adam,opt.Adamax , opt.NAdam, opt.SGD , opt.RMSprop]
#optimizer = torch.optim.Adam(model1.parameters(), lr = lrate) 
loss = nn.MSELoss()

In [ ]:
result_data = {
    'Optim': [],
    'LR': [],
    'Epoch': [],
    'Nodes': [],
    'Train RMSE':[],
    'Test RMSE':[]
}

#Final_results1 = pd.DataFrame(result_data)


In [ ]:
Final_results1 = pd.DataFrame(result_data)

for optim in func:
    for lr in lrate:
        for epochs in n_iters:
            for node in nodes:
            
                model1 = onelayermodel(node)
                model1.to(device)
                optimizer = optim(model1.parameters(), lr=lr)
                for i in range(epochs):
                    optimizer.zero_grad()
                    y_pred = model1(xtr)
                    l = loss(y_pred,ytr)
                    l.backward()
                    optimizer.step()
                    training_rmse = torch.sqrt(l)
                
                print(f'Optimizer = {optim}|lr = {lr}|epochs = {epochs}|nodes = {node}| Training RMSE = {training_rmse}')
            
                with torch.no_grad():
                    model1.eval()
                    y_pred1 = model1(xte)
                    l = loss(y_pred1,yte)
                    rmse = torch.sqrt(l)
                print(f'Optimizer = {optim}|lr = {lr}|epochs = {epochs}|nodes = {node}| Testing RMSE = {rmse}')
                
                #training_rmse = training_rmse.cpu()
                #rmse = rmse.cpu()
                
                #training_rmse = training_rmse.detach().numpy()
                #rmse = rmse.detach().numpy()
                
                
                new_row = {'Optim': optim,'LR': lr,'Epoch': epochs,'Nodes': node,'Train RMSE':training_rmse.cpu().detach().numpy(),'Test RMSE':rmse.cpu().detach().numpy()}
                Final_results1.loc[len(Final_results1)] = new_row
                #print(Final_results1)

In [ ]:
#Final_results1

In [ ]:
n = [[5,3],[7,5]]

In [ ]:
import pickle



In [ ]:
torch.manual_seed(123)
for optim in func:
    for lr in lrate:
        for epochs in n_iters:
            for node in n:
            
                model1 = twolayermodel(node[0],node[1])
                model1.to(device)
                optimizer = optim(model1.parameters(), lr=lr)
                for i in range(epochs):
                    optimizer.zero_grad()
                    y_pred = model1(xtr)
                    l = loss(y_pred,ytr)
                    l.backward()
                    optimizer.step()
                    training_rmse = torch.sqrt(l)
                
                print(f'Optimizer = {optim}|lr = {lr}|epochs = {epochs}|nodes = {node}| Training RMSE = {training_rmse}')
            
                with torch.no_grad():
                    model1.eval()
                    y_pred1 = model1(xte)
                    l = loss(y_pred1,yte)
                    rmse = torch.sqrt(l)
                print(f'Optimizer = {optim}|lr = {lr}|epochs = {epochs}|nodes = {node}| Testing RMSE = {rmse}')
                
                new_row = {'Optim': optim,'LR': lr,'Epoch': epochs,'Nodes': node,'Train RMSE':training_rmse.cpu().detach().numpy(),'Test RMSE':rmse.cpu().detach().numpy()}
                Final_results1.loc[len(Final_results1)] = new_row
                
                
                if optim==opt.Adamax and lr==0.001 and epochs ==25000 and node==n[1]:
                    # save the iris classification model as a pickle file
                    model_pkl_file = "U6DC_pred_final.pkl"
                    model_bro = model1.cpu()

                    with open(model_pkl_file, 'wb') as file:  
                        pickle.dump(model_bro, file)
                        
                    for weight in model1.parameters():
                        print(weight)
                    
                    pd.DataFrame(y_pred.cpu().detach().numpy()).to_excel('train_predictions.xlsx')
                    pd.DataFrame(y_pred1.cpu().detach().numpy()).to_excel('test_predictions.xlsx')
                    
                    

In [ ]:
Final_results1.to_excel('baskaroplms.xlsx')

In [ ]:
end = time.time()
print(f"Time taken : {end-start}")

In [ ]:
model1 = twolayermodel(7,5)
model1.to(device)
optimizer = opt.Adamax(model1.parameters(), lr=0.001)
for i in range(25000):
    optimizer.zero_grad()
    y_pred = model1(xtr)
    l = loss(y_pred,ytr)
    l.backward()
    optimizer.step()
    training_rmse = torch.sqrt(l)
                
print(f'Optimizer = Adamax|lr = 0.1|epochs = 25000|nodes = 7,5| Training RMSE = {training_rmse}')
            
with torch.no_grad():
    model1.eval()
    y_pred1 = model1(xte)
    l = loss(y_pred1,yte)
    rmse = torch.sqrt(l)
print(f'Optimizer = Adamax|lr = 0.1|epochs = 25000|nodes = 7,5| Testing RMSE = {rmse}')
                

In [ ]:
for weight in model1.parameters():
    print(weight)

In [ ]:
y = y_pred[:,1]
x = ytr[:,1]

y = y.cpu().detach().numpy()
x = x.cpu().detach().numpy()

#x = (x*(138.014-0.01438699))+0.01438699
#y = (y*(138.014-0.01438699))+0.01438699

x = (x*(0.234801988-0.011447075))+0.011447075
y = (y*(0.234801988-0.011447075))+0.011447075

plt.scatter(x,y, color = '#88c999')

x1 = np.linspace(0,0.3,100)
y1 = x1 
plt.plot(y1,x1)
plt.xlabel("exp")
plt.ylabel('pred')
plt.title('Train Hydrogen')
plt.show()

In [ ]:
yu = y_pred[:,0]
xu = ytr[:,0]

yu = yu.cpu().detach().numpy()
xu = xu.cpu().detach().numpy()

#x = (x*(138.014-0.01438699))+0.01438699
#y = (y*(138.014-0.01438699))+0.01438699

xu = (xu*(31.39216917-0.01438699))+0.01438699
yu = (yu*(31.39216917-0.01438699))+0.01438699

plt.scatter(xu,yu, color = '#88c999')

x1 = np.linspace(0,35,100)
y1 = x1 
plt.plot(y1,x1)
plt.xlabel("exp")
plt.ylabel('pred')
plt.title('Train Uranium')
plt.show()

In [ ]:
ytu = y_pred1[:,0]
xtu = yte[:,0]

ytu = ytu.cpu().detach().numpy()
xtu = xtu.cpu().detach().numpy()

#x = (x*(138.014-0.01438699))+0.01438699
#y = (y*(138.014-0.01438699))+0.01438699

xtu = (xtu*(31.39216917-0.01438699))+0.01438699
ytu = (ytu*(31.39216917-0.01438699))+0.01438699

plt.scatter(xtu,ytu, color = '#88c999')

x1 = np.linspace(0,33,100)
y1 = x1 
plt.plot(y1,x1)
plt.title('Test Uranium')
plt.xlabel("exp")
plt.ylabel('pred')
plt.show()